In [1]:
import pandas as pd
import numpy as np
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import pickle
from keras.preprocessing import sequence,text
from keras.models import load_model


Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
df = pd.read_csv("/kaggle/input/movie-review-sentiment-analysis-kernels-only/train.tsv", sep='\t')
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [3]:
df["Sentiment"].value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [4]:
print("Mean of word count in test set",np.mean(df['Phrase'].apply(lambda x: len(x.split()))))
print("Max of word count in test set",np.max(df['Phrase'].apply(lambda x: len(x.split()))))
print("Min of word count in test set",np.min(df['Phrase'].apply(lambda x: len(x.split()))))

Mean of word count in test set 7.203364090734333
Max of word count in test set 52
Min of word count in test set 0


In [5]:
results = set()
df['Phrase'].str.lower().str.split().apply(results.update)
print("Unique  words in phrases", len(results))

Unique  words in phrases 16531


In [6]:
x = df["Phrase"]
y = df["Sentiment"]

# trainx, testx, trainy, testy = train_test_split(x,y, test_size=0.33, random_state=42)

In [7]:
max_features = 17000
max_words = 55

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x)
x_train = tokenizer.texts_to_sequences(x)
# X_val = tokenizer.texts_to_sequences(X_val_text)
# X_test = tokenizer.texts_to_sequences(test_text)

x1 = sequence.pad_sequences(x_train, maxlen=max_words)
# X_val = sequence.pad_sequences(X_val, maxlen=max_words)
# X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # loading
# with open('tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)
target = to_categorical(y)

In [8]:
model=Sequential()
model.add(Embedding(max_features,250,mask_zero=True))
model.add(LSTM(128,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 250)         4250000   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         194048    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 4,493,781
Trainable params: 4,493,781
Non-trainable params: 0
_________________________________________________________________


In [9]:
%%time
history=model.fit(x1, target, epochs=5, batch_size=50, verbose=1)

Epoch 1/5
156060/156060 [==============================] - 673s 4ms/step - loss: 0.9555 - acc: 0.6146
Epoch 2/5
141750/156060 [==========================>...] - ETA: 1:00 - loss: 0.7727 - acc: 0.6800

In [10]:

# model.save('sentiment_model.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
# model = load_model('sentiment_model.h5')

In [11]:
df1 = pd.read_csv("/kaggle/input/movie-review-sentiment-analysis-kernels-only/test.tsv", sep="\t")
df1.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [12]:
testx = df1["Phrase"]

testx1 = tokenizer.texts_to_sequences(testx)
# X_val = tokenizer.texts_to_sequences(X_val_text)
# X_test = tokenizer.texts_to_sequences(test_text)

testx1 = sequence.pad_sequences(testx1, maxlen=max_words)


In [13]:
out1 = model.predict_classes(testx1)
out1

array([2, 2, 2, ..., 2, 2, 1])

In [14]:
x4 = pd.DataFrame({'PhraseId':np.array(df1["PhraseId"]), 'Sentiment':out1})
# x4["Sentiment"] = x4["Sentiment"].apply(lambda x:int(round(x)))
x4.to_csv("output.csv", index=False)